In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: SkiaSharp"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.6.3-alpha"
#r "nuget: System.Linq.Async"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;


In [ ]:
var settings  = Settings.LoadFromFile("config/settings.json");
#pragma warning disable SKEXP0001, SKEXP0020, SKEXP0010
var memory = new MemoryBuilder()
                    .WithAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", settings.azureEndpoint, settings.apiKey)
                    .WithMemoryStore(new AzureAISearchMemoryStore(settings.aisearchendpoint, settings.aisearchkey))
                    .Build();

In [ ]:
using System.Collections;
using System.Linq;

var gitHubFiles = new Dictionary<string, string>
{
    ["https://github.com/microsoft/semantic-kernel/blob/main/README.md"]
        = "README: Installation, getting started, and how to contribute",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb"]
        = "Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks//00-getting-started.ipynb"]
        = "Jupyter notebook describing how to get started with the Semantic Kernel",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/plugins/ChatPlugin/ChatGPT"]
        = "Sample demonstrating how to create a chat plugin interfacing with ChatGPT",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/SemanticKernel/Memory/VolatileMemoryStore.cs"]
        = "C# class that defines a volatile embedding store"
};


foreach (var entry in gitHubFiles)
{
    await memory.SaveReferenceAsync(
        collection: "GitHubFiles",
        externalSourceName: "GitHub",
        externalId: entry.Key,
        description: entry.Value,
        text: entry.Value);
}

In [ ]:
var question = new[]{"How do I get started?", "Can I build a chat with SK?"};
foreach(var q in question)
{
    Console.WriteLine($"Searching for: {q}");
    var results = await memory.SearchAsync("GitHubFiles", q, limit: 1).ToListAsync();
    var result = results.FirstOrDefault();
    if (result != null)
    {
        Console.WriteLine("URL:     : " + result.Metadata.Id);
        Console.WriteLine("Title    : " + result.Metadata.Description);
        Console.WriteLine("Relevance: " + result.Relevance);
    }
    Console.WriteLine();
}